In [1]:
import rasa_nlu
print(rasa_nlu.__version__)

0.12.3


In [2]:
#Importing required libraries
from rasa_nlu.training_data import load_data
from rasa_nlu import config
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from  rasa_nlu.model import Metadata, Interpreter
import pandas as pd
import numpy as np
import json
from pandas import*
import csv
import string
import re

In [3]:
import pandas as pd
import re
import jieba
from jieba import posseg as pseg

In [4]:
#input data for chinese messages
df = pd.read_excel("excel.xlsx", sheet_name= 'Sheet2')

In [7]:
#help function for message encoding
def encode_decode(sentence):
    sent1 = sentence.encode('Utf-8', 'replace')
    sent2 = sent1.decode('Utf-8', 'replace')
    return sent2

In [8]:
pattern = '[a-zA-Z0-9]'

In [9]:
#removing english words and digits
cleaned_data = []
for i in df.Mention:
    sent = re.sub(pattern , '', i)
    cleaned_data.append(encode_decode(sent))

In [10]:
df['Mention_cleaned'] = cleaned_data # todo : Need to remove punctuation marks 

In [11]:
def get_stop_words(): # load stopwords from txt file
    file = open('stopwords-zh.txt','rb').read().decode('utf8').split('\n')
    return set(file)

In [70]:
stopwords = list(get_stop_words())
punctuations = "！？。＂#＃$＄％&＆'＇()（）*＊+＋，-－/／:：;；<＜=＝>＞@[［＼\］] \
    ＾^＿_｀`{｛|｜}｝~～《》｟｠｢｣､、〃「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟 \
    〰〾〿–—‘’‛“”„‟…‧﹏."

In [43]:
# Function for tokenizing giant string(stemming, removing stopwords and punctuation)
def clean_sentence(sentence):
    word_list = jieba.tokenize(sentence)
    word_list = [word[0] for word in word_list if word[0] not in stopwords and word[0] not in punctuations]
    new_sent = " ".join(word_list)
    return new_sent  

In [14]:
# checking whether verb available in splitted sentence
def has_verb(sentence, verbose=False):
    pos_tagged = pseg.cut(sentence)
    if verbose:
        print(pos_tagged)
    if 'v' in [tag for word,tag in pos_tagged]:
        return True
    return False

In [15]:
#helper function for splitting sentences
def custom_sentence_tokenizer(text, verbose=False):
    # container for final result
    new_split = []
    
    # Split sentences by a comma, 'and' and 'or'.
    text_list = re.split('[。| ！| ？| 」]+', text) #[。| ！| ？| 」]+\s
    
    # Remove white spaces and empty string elements from the list
    text_list = [x.strip() for x in text_list]
    #text_list = list(filter(None, text_list))
        
    # Append first list element to the new list.
    new_split.append(text_list[0])
    
    # Check if the splits are valid sentences. If not, glue the parts together again.
    for index in range(1, len(text_list)):
        
        # Keep the split if both parts of the sentences contain a verb.
        if has_verb(text_list[index-1], verbose) and has_verb(text_list[index], verbose):
            new_split.append(text_list[index])
        # Glue the parts together again, since POS requirements are not met.
        else:
            new_split[-1] += ' ' + text_list[index]
    
    if verbose:
        print('[.] Input sentence:')
        print('    ', text)
        print('[.] Output sentence(s):')
        print('    ', new_split)
    return new_split

In [81]:
#helper function for splitting sentences
def split_sentences_in_dataframe(df, verbose=False):
    new_df = pd.DataFrame()
    for index, row in df.iterrows():
        text = row['Mention_cleaned']
        text_list = custom_sentence_tokenizer(text, verbose)
        for text_part in text_list:
            new_row = row.copy()
            clean_text_part = clean_sentence(text_part)
            if(clean_text_part!= ''):
                new_row['Mention_cleaned'] = clean_text_part
                new_df = new_df.append(new_row)
    return new_df

In [82]:
df1 = split_sentences_in_dataframe(df)

In [85]:
training_data = df1[['Mention','Mention_cleaned','Intent']] #pd.DataFrame(columns=['Mention','Mention_cleaned','Intent'])

In [87]:
#Convert excel training data into rasa_nlu format
def convert_excel_rasa_nlu(training_data):
    results = []
    common_examples_list = []
    common_examples = []
    x = []
    for index, row in training_data.iterrows():
        x = (row['Intent'],row['Mention_cleaned'],[])
        results.append(x)
    for result in results:
        keys = ['intent', 'text', 'entities']
        common_examples.append(dict(zip(keys, result)))
    rasa_nlu_data1 = {"regex_features" : [], "entity_synonyms": [],"common_examples": []}
    rasa_nlu_data1["common_examples"] = common_examples
    dataformat = dict(rasa_nlu_data = rasa_nlu_data1)
    return(dataformat)

In [88]:
import json
with open('rasa_dataset_chinese.json', 'w') as fp:
    json.dump(convert_excel_rasa_nlu(training_data),fp)

In [89]:
#training rasa model
def train_nlu(data, configs, model_dir):
    #print('in')
    training_data = load_data(data)
    trainer = Trainer(config.load(configs))
    trainer.train(training_data)
    model_directory = trainer.persist(model_dir, fixed_model_name = 'rasaTLnlu')

In [90]:
# will take few minutes
if __name__ == '__main__':
    train_nlu('rasa_dataset_chinese.json', 'config_rasa_chinese.yaml', './models/')
    interpreter = Interpreter.load('./models/default/rasaTLnlu')

C:\Users\Jhansi\Anaconda3\lib\site-packages\rasa_nlu\training_data\training_data.py:186: UserWarning: Intent 'temperature' has only 1 training examples! Minimum is 2, training may fail.
  self.MIN_EXAMPLES_PER_INTENT))
C:\Users\Jhansi\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.
INFO:tensorflow:Restoring parameters from ./models/default/rasaTLnlu\intent_classifier_tensorflow_embedding.ckpt


In [92]:
# test data to predict intent of the message
df_test = pd.read_excel("test_excel.xlsx")

In [94]:
cleaned_data = []
for i in df_test.Mention:
    sent = re.sub(pattern , '', i)
    cleaned_data.append(encode_decode(sent))

In [95]:
df_test['Mention_cleaned'] = cleaned_data

In [96]:
df_test_1 = split_sentences_in_dataframe(df_test)

In [97]:
df_test_1.id = df_test_1.id.astype('int')

In [98]:
# predicting the intent of message
def predict_text_intent(intent_result, id_pred):
    #global df_rasa_output
    if intent_result['intent']['name'] in ['spam']:
        #check if it is a spam comment, if yes then append into the final dataframe
        df1 = pd.DataFrame(columns=['id','mention','confidence','intent'])
        extract_text = intent_result['text']
        extract_confidence = intent_result['intent']['confidence']
        extract_intent = intent_result['intent']['name']
        if extract_confidence > 0.5:
            row = [id_pred, extract_text, extract_confidence, extract_intent]
            df1.loc[len(df1)] = row
            df1.append(row, ignore_index=True)
        #df_rasa_output = df_rasa_output.append(pd.DataFrame(data = df1), ignore_index=True)
    else:
        df1 = pd.DataFrame(columns=['id','mention','confidence','intent'])
        count_intent_range = len(intent_result['intent_ranking'])
        intent_ranking_result = intent_result['intent_ranking']
        for i in range(count_intent_range):
            string = []
            index_range = i
            string = intent_ranking_result[index_range]
            extract_text = intent_result['text']
            extract_confidence = string['confidence']
            extract_intent = string['name']
            if extract_confidence > 0.5:
                row = [id_pred, extract_text, extract_confidence, extract_intent]
                df1.loc[len(df1)] = row
                df1.append(row, ignore_index=True)
            #df_rasa_output = df_rasa_output.append(pd.DataFrame(data = df1), ignore_index=True)
    return df1#df_rasa_output.drop_duplicates(inplace=True)

In [99]:
test_dataset_count = df_test_1.Mention_cleaned.count()
df_rasa_output = pd.DataFrame(columns=['id','mention','confidence','intent'])
for i in range(test_dataset_count):
    extract_chatlog = df_test_1.Mention_cleaned.iloc[i]
    if extract_chatlog != '':
        classify_chat = interpreter.parse(extract_chatlog)
        id_pred = df_test_1.id.iloc[i]
        df1 = predict_text_intent(classify_chat, id_pred)    
        df_rasa_output = df_rasa_output.append(pd.DataFrame(data = df1), ignore_index=True)

C:\Users\Jhansi\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
